In [1]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec

C:\Users\Atul Rawat\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\Atul Rawat\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https

In [2]:
! pip install rank_bm25
import nltk 
import string 
import re 
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Atul
[nltk_data]     Rawat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from rank_bm25 import BM25Okapi
dataset2 = pd.read_csv('wiki_voyage_correct.csv')
dataset1 = pd.read_csv('final_dataset_wo_duplicates.csv')

In [4]:
print(dataset2['wiki voyage'][0])

         28.6377 77.2058     From Wikivoyage  Asia > South Asia > India > Plains (India) > Delhi    Delhi       Contents   1  Understand   1.1  History  1.2  Demographics  1.3  Orientation  1.4  Climate  1.5  Read    2  Districts  3  Talk  4  Get in   4.1  By plane   4.1.1  Airlines  4.1.2  To travel between the airport and the city    4.2  Other airports  4.3  By bus  4.4  By train   4.4.1  Introduction  4.4.2  Stations & ticket offices      5  Get around   5.1  By metro  5.2  By local train  5.3  By bus   5.3.1  Hop on Hop off    5.4  By taxi   5.4.1  Transportation Network Taxis    5.5  By auto rickshaws  5.6  By cycle rickshaws  5.7  By Electric Rickshaws  5.8  On foot    6  See  7  Do  8  Learn  9  Work  10  Buy   10.1  Clothing  10.2  Computers  10.3  Books  10.4  Tea    11  Eat   11.1  Budget  11.2  Mid-range  11.3  Splurge   11.3.1  Barbeque/grills  11.3.2  Japanese  11.3.3  Middle Eastern  11.3.4  Thai  11.3.5  Tibetan  11.3.6  Chinese  11.3.7  Korean  11.3.8  Afghani  11.3.9 

In [7]:
def word2vec_sim_score(q,doc_list):
  if len(doc_list) ==287:
    doc_list = doc_list[:-1]
  doc_list.append(q)
  stemmed_doc_list, doc_vector = list(), list()
  for doc in doc_list:
    stemmed_doc_list.append(doc.split(" "))
  # print(stemmed_doc_list)
  word2vec = Word2Vec(stemmed_doc_list,min_count=1,vector_size=1000)
  for index,row in enumerate(stemmed_doc_list):
    model_vector = (np.mean([word2vec[token] for token in row], axis=0)).tolist()
    doc_vector.append(model_vector)
  sim = cosine_similarity(doc_vector)
  return np.asarray(sim[-1][:-1])

def tf_idf_sim_score(q,doc_list):
  tfvec = TfidfVectorizer(stop_words='english')
  if len(doc_list) ==287:
    doc_list = doc_list[:-1]
  doc_list.append(q)
  vec_corpus = tfvec.fit_transform(doc_list)
  sim = cosine_similarity(vec_corpus)
  return np.asarray(sim[-1][:-1])

def countvec_sim_score(q,doc_list):
  countvec = CountVectorizer(stop_words='english')
  if len(doc_list) ==287:
    doc_list = doc_list[:-1]
  doc_list.append(q)
  vec_corpus = countvec.fit_transform(doc_list)
  sim = cosine_similarity(vec_corpus)
  return np.asarray(sim[-1][:-1])

In [8]:
document_list=[]
lemmatizer = WordNetLemmatizer() 
i=0
cities = dataset2['City']
for i in range(dataset2.shape[0]):
  k= cities[i]
  document = ' description of '+ k + dataset2.iloc[i]['description']+'Places to visit in '+k  +str(dataset2.iloc[i]['sites']) + '\n how to reach '+ k+str(dataset2.iloc[i]['how to reach']) + '\n best time to visit'+k+dataset2.iloc[i]['best to time visit'] + ' '+dataset2.iloc[i]['wiki voyage']
  document = document.lower()
  input_str = document
  translator = str.maketrans('', '', string.punctuation) 
  document = document.translate(translator) 
  document=lemmatizer.lemmatize(document)

  document_list.append(document)
  i+=1

total_tags = []
total_tags.append(dataset1['historical & heritage'].value_counts())
total_tags.append(dataset1['city'].value_counts())
total_tags.append(dataset1['pilgrimage'].value_counts())
tags_hills = dataset1['hill station'].value_counts()
total_tags.append(tags_hills)
tags_beach = dataset1['beach'].value_counts()
total_tags.append(tags_beach)
tags_lakes = dataset1['lake & backwater'].value_counts()
total_tags.append(tags_lakes)
tags_adventure = dataset1['adventure / trekking'].value_counts()
total_tags.append(tags_adventure)
tags_wildlife = dataset1['wildlife'].value_counts()
total_tags.append(tags_wildlife)
tags_waterfall = dataset1['waterfall'].value_counts()
total_tags.append(tags_waterfall)
tags_nature = dataset1['nature & scenic'].value_counts()
total_tags.append(tags_nature)


query = input('Enter query related to tourism : ')

corpus = document_list
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)
query=query.lower()
query=lemmatizer.lemmatize(query)
print(query)

#-------Change here--------
recommender = int(input('Enter the recommender to use \n 1. BM25 \t 2.TF-IDF+Cosine similarity \n 3.Count Vectorizer + Cosine Similarity \t 4. Word2Vec + Cosine Similarity \t'))
if(recommender == 1):
  tokenized_query = query.split(" ")
  print('Using BM25')
  doc_scores = bm25.get_scores(tokenized_query)
if(recommender == 2):
  score = tf_idf_sim_score(query,document_list)
  print('Using TF-IDF')
  doc_scores = score
if(recommender == 3):
  score = countvec_sim_score(query,document_list)
  print('Using Count Vec')
  doc_scores = score
if(recommender == 4):
  print('Using Word2Vec')
  doc_scores = word2vec_sim_score(query,document_list)

print('-----------------------------------------------------------------------------')
#------------------------------

a= doc_scores
recommendations = int(input('Enter number of recommendations you want : '))
recommendations_index =a.argsort()[-recommendations:][::-1]
cities = dataset2['City']
desc = dataset2['description']
print('ID','     ','Place',   '    Description')
print()
for i in range(recommendations_index.shape[0]):
  print(recommendations_index[i],'  ', cities[recommendations_index[i]], ' : ',desc[recommendations_index[i]])

selected_recommendation = int(input('\n Enter the id of place you are interested in :'))
row = dataset2.iloc[selected_recommendation]
print(' \n Place Description :',row['description'])
print()
print('Near by places to visit :\n',row['sites'])
print()
print(' How to reach :',row['how to reach'])
print()
print('Best time to visit',row['best to time visit'])

Enter query related to tourism : hills
hill
Enter the recommender to use 
 1. BM25 	 2.TF-IDF+Cosine similarity 
 3.Count Vectorizer + Cosine Similarity 	 4. Word2Vec + Cosine Similarity 	4
Using Word2Vec


TypeError: 'Word2Vec' object is not subscriptable

In [ ]:
stemmed_doc_list, doc_vector = list(), list()
for doc in doc_list:
  stemmed_doc_list.append(doc.split(" "))
  # print(stemmed_doc_list)
word2vec = Word2Vec(stemmed_doc_list,min_count=1,size=1000)
for index,row in enumerate(stemmed_doc_list):
  model_vector = (np.mean([word2vec[token] for token in row], axis=0)).tolist()
  doc_vector.append(model_vector)
sim = cosine_similarity(doc_vector)

In [10]:
stemmed_doc_list, doc_vector = list(), list()

In [11]:
for doc in document_list:
  stemmed_doc_list.append(doc.split(" "))

NameError: ignored